# Module 9: 圖神經網路 (GNN)

## 學習目標
- 理解圖結構資料的特點
- 掌握訊息傳遞機制 (Message Passing)
- 實現 GCN 和 GAT
- 使用 PyTorch Geometric 進行節點分類

## 為什麼要學 GNN？

許多真實世界的資料天然具有**圖結構**：
- 社交網路（人與人的關係）
- 分子結構（原子與化學鍵）
- 知識圖譜（實體與關係）
- 推薦系統（用戶-物品互動）

傳統神經網路假設資料是獨立的向量或網格，無法處理這種關係結構。

In [ ]:
# 安裝 PyTorch Geometric（如果尚未安裝）
# !pip install torch-geometric

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

---
## Part 1: 圖的基本概念

### 圖的組成

一個圖 $G = (V, E)$ 由以下元素組成：
- **節點 (Nodes/Vertices)** $V$：圖中的實體
- **邊 (Edges)** $E$：節點之間的連接
- **節點特徵** $X$：每個節點的屬性向量
- **邊特徵**（可選）：邊的屬性

```
簡單圖示例：
    
    (A) ─── (B)
     │ \   / │
     │  \ /  │
     │   X   │
     │  / \  │
     │ /   \ │
    (C) ─── (D)

節點: {A, B, C, D}
邊: {(A,B), (A,C), (A,D), (B,C), (B,D), (C,D)}
```

In [ ]:
def visualize_graph_basics():
    """
    可視化圖的基本結構
    """
    # 建立一個簡單的圖
    G = nx.karate_club_graph()  # 著名的空手道俱樂部社交網路
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 圖結構可視化
    pos = nx.spring_layout(G, seed=42)
    
    # 根據社群上色
    clubs = [G.nodes[i]['club'] for i in G.nodes()]
    colors = ['#FF6B6B' if c == 'Mr. Hi' else '#4ECDC4' for c in clubs]
    
    nx.draw(G, pos, ax=axes[0], node_color=colors, 
            with_labels=True, node_size=300, font_size=8)
    axes[0].set_title("空手道俱樂部社交網路")
    
    # 鄰接矩陣可視化
    adj_matrix = nx.adjacency_matrix(G).todense()
    im = axes[1].imshow(adj_matrix, cmap='Blues')
    axes[1].set_title("鄰接矩陣 A")
    axes[1].set_xlabel("節點 j")
    axes[1].set_ylabel("節點 i")
    plt.colorbar(im, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
    print(f"節點數: {G.number_of_nodes()}")
    print(f"邊數: {G.number_of_edges()}")
    print(f"平均度數: {np.mean([d for n, d in G.degree()]):.2f}")

visualize_graph_basics()

### 圖的表示方式

在深度學習中，圖通常用以下方式表示：

1. **鄰接矩陣 A**：$A_{ij} = 1$ 如果節點 i 和 j 相連
2. **邊列表**：所有邊的列表 [(i, j), ...]
3. **節點特徵矩陣 X**：$X \in \mathbb{R}^{N \times F}$，N 個節點，每個 F 維特徵

In [ ]:
# 三種表示方式
def graph_representations():
    """
    展示圖的不同表示方式
    """
    # 簡單的 4 節點圖
    # 0 -- 1
    # |    |
    # 2 -- 3
    
    num_nodes = 4
    
    # 1. 鄰接矩陣
    adj_matrix = torch.tensor([
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 0]
    ], dtype=torch.float32)
    
    # 2. 邊列表 (COO 格式，PyTorch Geometric 常用)
    edge_index = torch.tensor([
        [0, 0, 1, 1, 2, 2, 3, 3],  # 源節點
        [1, 2, 0, 3, 0, 3, 1, 2]   # 目標節點
    ], dtype=torch.long)
    
    # 3. 節點特徵
    node_features = torch.randn(num_nodes, 8)  # 4 個節點，每個 8 維特徵
    
    print("1. 鄰接矩陣 A:")
    print(adj_matrix)
    print(f"\n2. 邊列表 (edge_index):")
    print(f"   源節點:   {edge_index[0].tolist()}")
    print(f"   目標節點: {edge_index[1].tolist()}")
    print(f"\n3. 節點特徵 X: 形狀 {node_features.shape}")
    
    return adj_matrix, edge_index, node_features

adj_matrix, edge_index, node_features = graph_representations()

---
## Part 2: 訊息傳遞機制 (Message Passing)

### 核心思想

GNN 的核心是**訊息傳遞**：每個節點通過聚合鄰居的資訊來更新自己的表示。

```
訊息傳遞三步驟：

1. 訊息 (Message): 每個鄰居發送資訊
   m_{j→i} = MSG(h_j)

2. 聚合 (Aggregate): 收集所有鄰居的訊息
   m_i = AGG({m_{j→i} : j ∈ N(i)})

3. 更新 (Update): 用聚合的訊息更新節點表示
   h'_i = UPDATE(h_i, m_i)

圖示:
        ┌─────┐
    ┌───│ 鄰居1│───┐
    │   └─────┘   │
    │             │
    ▼   ┌─────┐   ▼
    └───│ 節點 │───┘ ← 聚合鄰居訊息
        └─────┘
            │
            ▼
        更新表示
```

In [ ]:
class SimpleMessagePassing(nn.Module):
    """
    最簡單的訊息傳遞層
    
    聚合方式：平均鄰居特徵
    更新方式：線性變換
    """
    
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x, adj):
        """
        Args:
            x: 節點特徵 [N, F]
            adj: 鄰接矩陣 [N, N]
        """
        # 計算度數矩陣（每個節點的鄰居數）
        degree = adj.sum(dim=1, keepdim=True) + 1e-6
        
        # 聚合：鄰居特徵的平均
        # adj @ x 對每個節點求鄰居特徵的和
        neighbor_sum = torch.matmul(adj, x)
        neighbor_mean = neighbor_sum / degree
        
        # 更新：線性變換 + 非線性
        out = self.linear(neighbor_mean)
        
        return F.relu(out)

# 測試
layer = SimpleMessagePassing(8, 16)
out = layer(node_features, adj_matrix)
print(f"輸入形狀: {node_features.shape}")
print(f"輸出形狀: {out.shape}")

---
## Part 3: GCN (Graph Convolutional Network)

### GCN 的直觀理解

GCN 是最經典的 GNN，它的更新公式是：

$$H^{(l+1)} = \sigma(\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)})$$

其中：
- $\tilde{A} = A + I$（加入自連接）
- $\tilde{D}$ 是 $\tilde{A}$ 的度數矩陣
- $W^{(l)}$ 是可學習的權重

**簡化理解**：對每個節點，取自己和鄰居特徵的加權平均，然後做線性變換。

In [ ]:
class GCNLayer(nn.Module):
    """
    Graph Convolutional Layer
    
    實現標準的 GCN 層
    """
    
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
    
    def forward(self, x, adj):
        """
        GCN 前向傳播
        
        H' = D^(-1/2) A D^(-1/2) X W
        """
        # 加入自連接
        adj_hat = adj + torch.eye(adj.size(0)).to(adj.device)
        
        # 計算度數矩陣的逆平方根
        degree = adj_hat.sum(dim=1)
        d_inv_sqrt = torch.diag(degree.pow(-0.5))
        
        # 對稱歸一化: D^(-1/2) A D^(-1/2)
        adj_norm = torch.mm(torch.mm(d_inv_sqrt, adj_hat), d_inv_sqrt)
        
        # 聚合鄰居特徵
        support = torch.mm(x, self.weight)
        output = torch.mm(adj_norm, support)
        
        if self.bias is not None:
            output = output + self.bias
        
        return output

class GCN(nn.Module):
    """
    兩層 GCN 用於節點分類
    """
    
    def __init__(self, n_features, n_hidden, n_classes, dropout=0.5):
        super().__init__()
        self.gc1 = GCNLayer(n_features, n_hidden)
        self.gc2 = GCNLayer(n_hidden, n_classes)
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

# 測試
model = GCN(n_features=8, n_hidden=16, n_classes=2)
out = model(node_features, adj_matrix)
print(f"GCN 輸出形狀: {out.shape}")
print(f"預測類別機率:\n{torch.exp(out)}")

---
## Part 4: GAT (Graph Attention Network)

### GAT 的改進

GCN 對所有鄰居一視同仁，但實際上不同鄰居的重要性可能不同。

GAT 使用**注意力機制**為每個鄰居分配不同的權重：

$$\alpha_{ij} = \frac{\exp(\text{LeakyReLU}(a^T [Wh_i || Wh_j]))}{\sum_{k \in N(i)} \exp(\text{LeakyReLU}(a^T [Wh_i || Wh_k]))}$$

$$h'_i = \sigma\left(\sum_{j \in N(i)} \alpha_{ij} W h_j\right)$$

In [ ]:
class GATLayer(nn.Module):
    """
    Graph Attention Layer
    """
    
    def __init__(self, in_features, out_features, n_heads=1, dropout=0.6, concat=True):
        super().__init__()
        
        self.n_heads = n_heads
        self.concat = concat
        self.out_features = out_features
        
        # 每個注意力頭的變換矩陣
        self.W = nn.Parameter(torch.FloatTensor(n_heads, in_features, out_features))
        
        # 注意力向量
        self.a = nn.Parameter(torch.FloatTensor(n_heads, 2 * out_features, 1))
        
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(dropout)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W)
        nn.init.xavier_uniform_(self.a)
    
    def forward(self, x, adj):
        """
        Args:
            x: 節點特徵 [N, F]
            adj: 鄰接矩陣 [N, N]
        """
        N = x.size(0)
        
        # 線性變換: [N, F] -> [n_heads, N, out_features]
        h = torch.einsum('nf,hfo->hno', x, self.W)
        
        # 計算注意力係數
        # 建立所有節點對的特徵拼接
        # [n_heads, N, out_features] -> [n_heads, N, N, 2*out_features]
        h_repeat_i = h.unsqueeze(2).repeat(1, 1, N, 1)  # [h, N, N, F]
        h_repeat_j = h.unsqueeze(1).repeat(1, N, 1, 1)  # [h, N, N, F]
        concat_features = torch.cat([h_repeat_i, h_repeat_j], dim=-1)  # [h, N, N, 2F]
        
        # 計算注意力分數
        e = self.leaky_relu(torch.einsum('hnmf,hfo->hnm', concat_features, self.a).squeeze(-1))
        
        # 遮罩非鄰居
        adj_with_self = adj + torch.eye(N).to(adj.device)
        mask = (adj_with_self == 0)
        e = e.masked_fill(mask.unsqueeze(0), float('-inf'))
        
        # Softmax 歸一化
        attention = F.softmax(e, dim=-1)  # [n_heads, N, N]
        attention = self.dropout(attention)
        
        # 應用注意力權重聚合特徵
        h_prime = torch.einsum('hnm,hmo->hno', attention, h)  # [n_heads, N, out_features]
        
        # 合併多頭
        if self.concat:
            # 拼接所有頭: [N, n_heads * out_features]
            return h_prime.permute(1, 0, 2).contiguous().view(N, -1)
        else:
            # 平均所有頭: [N, out_features]
            return h_prime.mean(dim=0)

class GAT(nn.Module):
    """
    兩層 GAT 用於節點分類
    """
    
    def __init__(self, n_features, n_hidden, n_classes, n_heads=4, dropout=0.6):
        super().__init__()
        
        self.gat1 = GATLayer(n_features, n_hidden, n_heads=n_heads, dropout=dropout, concat=True)
        self.gat2 = GATLayer(n_hidden * n_heads, n_classes, n_heads=1, dropout=dropout, concat=False)
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.gat1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gat2(x, adj)
        return F.log_softmax(x, dim=1)

# 測試
gat_model = GAT(n_features=8, n_hidden=8, n_classes=2, n_heads=4)
out = gat_model(node_features, adj_matrix)
print(f"GAT 輸出形狀: {out.shape}")

---
## Part 5: 使用 PyTorch Geometric

PyTorch Geometric (PyG) 是專門用於圖神經網路的庫，提供了許多預實現的層和資料集。

In [ ]:
# 嘗試導入 PyG（如果有安裝的話）
try:
    from torch_geometric.datasets import Planetoid
    from torch_geometric.nn import GCNConv, GATConv
    from torch_geometric.transforms import NormalizeFeatures
    HAS_PYG = True
    print("PyTorch Geometric 已載入")
except ImportError:
    HAS_PYG = False
    print("PyTorch Geometric 未安裝，將使用自定義實現")

In [ ]:
if HAS_PYG:
    # 載入 Cora 資料集
    dataset = Planetoid(root='./data', name='Cora', transform=NormalizeFeatures())
    data = dataset[0]
    
    print(f"資料集: {dataset}")
    print(f"節點數: {data.num_nodes}")
    print(f"邊數: {data.num_edges}")
    print(f"特徵維度: {data.num_features}")
    print(f"類別數: {dataset.num_classes}")
    print(f"訓練節點: {data.train_mask.sum().item()}")
    print(f"驗證節點: {data.val_mask.sum().item()}")
    print(f"測試節點: {data.test_mask.sum().item()}")
else:
    # 建立模擬資料
    print("使用模擬資料")
    G = nx.karate_club_graph()
    
    # 準備資料
    num_nodes = G.number_of_nodes()
    num_features = 34  # 與節點數相同的特徵（one-hot）
    num_classes = 2
    
    # 節點特徵：one-hot 編碼
    x = torch.eye(num_nodes)
    
    # 標籤：俱樂部歸屬
    y = torch.tensor([0 if G.nodes[i]['club'] == 'Mr. Hi' else 1 for i in range(num_nodes)])
    
    # 邊列表
    edges = list(G.edges())
    edge_index = torch.tensor([[e[0] for e in edges] + [e[1] for e in edges],
                               [e[1] for e in edges] + [e[0] for e in edges]], dtype=torch.long)
    
    # 鄰接矩陣
    adj = torch.zeros(num_nodes, num_nodes)
    for i, j in edges:
        adj[i, j] = 1
        adj[j, i] = 1
    
    # 訓練/測試分割
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[:20] = True
    test_mask = ~train_mask
    
    print(f"節點數: {num_nodes}")
    print(f"邊數: {len(edges)}")

In [ ]:
if HAS_PYG:
    class PyGGCN(nn.Module):
        """使用 PyG 的 GCN"""
        def __init__(self, n_features, n_hidden, n_classes, dropout=0.5):
            super().__init__()
            self.conv1 = GCNConv(n_features, n_hidden)
            self.conv2 = GCNConv(n_hidden, n_classes)
            self.dropout = dropout
            
        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
    
    class PyGGAT(nn.Module):
        """使用 PyG 的 GAT"""
        def __init__(self, n_features, n_hidden, n_classes, n_heads=4, dropout=0.6):
            super().__init__()
            self.conv1 = GATConv(n_features, n_hidden, heads=n_heads, dropout=dropout)
            self.conv2 = GATConv(n_hidden * n_heads, n_classes, heads=1, concat=False, dropout=dropout)
            self.dropout = dropout
            
        def forward(self, x, edge_index):
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
    
    # 訓練函數
    def train_pyg_model(model, data, epochs=200, lr=0.01, weight_decay=5e-4):
        model = model.to(device)
        data = data.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        
        best_val_acc = 0
        history = {'train_loss': [], 'val_acc': [], 'test_acc': []}
        
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            
            out = model(data.x, data.edge_index)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            
            loss.backward()
            optimizer.step()
            
            # 評估
            model.eval()
            with torch.no_grad():
                out = model(data.x, data.edge_index)
                pred = out.argmax(dim=1)
                
                val_acc = (pred[data.val_mask] == data.y[data.val_mask]).sum() / data.val_mask.sum()
                test_acc = (pred[data.test_mask] == data.y[data.test_mask]).sum() / data.test_mask.sum()
            
            history['train_loss'].append(loss.item())
            history['val_acc'].append(val_acc.item())
            history['test_acc'].append(test_acc.item())
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
            
            if (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")
        
        return history
    
    # 訓練 GCN
    print("\n訓練 GCN:")
    gcn_pyg = PyGGCN(dataset.num_features, 16, dataset.num_classes)
    gcn_history = train_pyg_model(gcn_pyg, data, epochs=200)
    
    # 訓練 GAT
    print("\n訓練 GAT:")
    gat_pyg = PyGGAT(dataset.num_features, 8, dataset.num_classes, n_heads=8)
    gat_history = train_pyg_model(gat_pyg, data, epochs=200)
    
else:
    # 使用自定義實現
    def train_custom_model(model, x, adj, y, train_mask, test_mask, epochs=200, lr=0.01):
        model = model.to(device)
        x = x.to(device)
        adj = adj.to(device)
        y = y.to(device)
        train_mask = train_mask.to(device)
        test_mask = test_mask.to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
        
        history = {'train_loss': [], 'test_acc': []}
        
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            
            out = model(x, adj)
            loss = F.nll_loss(out[train_mask], y[train_mask])
            
            loss.backward()
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                out = model(x, adj)
                pred = out.argmax(dim=1)
                test_acc = (pred[test_mask] == y[test_mask]).sum() / test_mask.sum()
            
            history['train_loss'].append(loss.item())
            history['test_acc'].append(test_acc.item())
            
            if (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}")
        
        return history
    
    print("\n訓練 GCN (自定義實現):")
    gcn_custom = GCN(num_features, 16, num_classes)
    gcn_history = train_custom_model(gcn_custom, x, adj, y, train_mask, test_mask, epochs=200)

In [ ]:
# 繪製訓練曲線
if HAS_PYG:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(gcn_history['train_loss'], label='GCN')
    axes[0].plot(gat_history['train_loss'], label='GAT')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Training Loss')
    axes[0].legend()
    axes[0].set_title('訓練損失')
    
    axes[1].plot(gcn_history['test_acc'], label='GCN')
    axes[1].plot(gat_history['test_acc'], label='GAT')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Test Accuracy')
    axes[1].legend()
    axes[1].set_title('測試準確率')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n最終測試準確率:")
    print(f"  GCN: {gcn_history['test_acc'][-1]*100:.2f}%")
    print(f"  GAT: {gat_history['test_acc'][-1]*100:.2f}%")

---
## 練習題

### 練習 1: GraphSAGE 層

**目標**: 實現 GraphSAGE 的採樣聚合

In [ ]:
# 練習 1: GraphSAGE

class GraphSAGELayer(nn.Module):
    """
    GraphSAGE 層
    
    與 GCN 的差異：
    1. 明確分開自己和鄰居的表示
    2. 可以使用不同的聚合函數（mean, max, LSTM 等）
    """
    
    def __init__(self, in_features, out_features, aggregator='mean'):
        super().__init__()
        
        self.aggregator = aggregator
        
        # 自己特徵的變換
        self.W_self = nn.Linear(in_features, out_features, bias=False)
        
        # 鄰居特徵的變換
        self.W_neighbor = nn.Linear(in_features, out_features, bias=False)
        
        self.bias = nn.Parameter(torch.zeros(out_features))
        
    def forward(self, x, adj):
        """
        h' = W_self * h + W_neighbor * AGG(h_neighbors)
        """
        # 聚合鄰居
        degree = adj.sum(dim=1, keepdim=True).clamp(min=1)
        
        if self.aggregator == 'mean':
            neighbor_sum = torch.mm(adj, x)
            neighbor_agg = neighbor_sum / degree
        elif self.aggregator == 'max':
            # 使用遮罩實現 max pooling
            mask = (adj > 0).float()
            x_expanded = x.unsqueeze(0).expand(x.size(0), -1, -1)  # [N, N, F]
            masked = x_expanded * mask.unsqueeze(-1) - (1 - mask.unsqueeze(-1)) * 1e9
            neighbor_agg = masked.max(dim=1)[0]
        else:
            neighbor_sum = torch.mm(adj, x)
            neighbor_agg = neighbor_sum / degree
        
        # 結合自己和鄰居的表示
        self_transformed = self.W_self(x)
        neighbor_transformed = self.W_neighbor(neighbor_agg)
        
        out = self_transformed + neighbor_transformed + self.bias
        
        # L2 歸一化（GraphSAGE 特有）
        out = F.normalize(out, p=2, dim=1)
        
        return out

class GraphSAGE(nn.Module):
    def __init__(self, n_features, n_hidden, n_classes, aggregator='mean'):
        super().__init__()
        self.sage1 = GraphSAGELayer(n_features, n_hidden, aggregator)
        self.sage2 = GraphSAGELayer(n_hidden, n_classes, aggregator)
        
    def forward(self, x, adj):
        x = F.relu(self.sage1(x, adj))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.sage2(x, adj)
        return F.log_softmax(x, dim=1)

# 測試
sage_model = GraphSAGE(8, 16, 2, aggregator='mean')
out = sage_model(node_features, adj_matrix)
print(f"GraphSAGE 輸出形狀: {out.shape}")

### 練習 2: 圖級別預測

**目標**: 實現圖級別的分類（而非節點級別）

In [ ]:
# 練習 2: 圖分類

class GraphClassifier(nn.Module):
    """
    圖級別分類器
    
    步驟：
    1. 用 GNN 層學習節點表示
    2. 使用 Readout 函數聚合所有節點表示
    3. 用 MLP 進行分類
    """
    
    def __init__(self, n_features, n_hidden, n_classes, readout='mean'):
        super().__init__()
        
        self.readout = readout
        
        # GNN 層
        self.conv1 = GCNLayer(n_features, n_hidden)
        self.conv2 = GCNLayer(n_hidden, n_hidden)
        self.conv3 = GCNLayer(n_hidden, n_hidden)
        
        # 分類器
        self.classifier = nn.Sequential(
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(n_hidden, n_classes)
        )
        
    def graph_readout(self, x):
        """
        將節點表示聚合為圖表示
        """
        if self.readout == 'mean':
            return x.mean(dim=0, keepdim=True)
        elif self.readout == 'max':
            return x.max(dim=0, keepdim=True)[0]
        elif self.readout == 'sum':
            return x.sum(dim=0, keepdim=True)
        else:
            return x.mean(dim=0, keepdim=True)
    
    def forward(self, x, adj):
        # 節點嵌入
        x = F.relu(self.conv1(x, adj))
        x = F.relu(self.conv2(x, adj))
        x = F.relu(self.conv3(x, adj))
        
        # 圖級別表示
        graph_embedding = self.graph_readout(x)
        
        # 分類
        out = self.classifier(graph_embedding)
        return out

# 測試
graph_clf = GraphClassifier(8, 32, 2, readout='mean')
out = graph_clf(node_features, adj_matrix)
print(f"圖分類輸出形狀: {out.shape}")
print(f"預測: {F.softmax(out, dim=1)}")

### 練習 3: 可視化注意力權重

**目標**: 可視化 GAT 的注意力權重

In [ ]:
# 練習 3: 注意力可視化

class GATWithAttention(nn.Module):
    """
    GAT 層，同時返回注意力權重
    """
    
    def __init__(self, in_features, out_features):
        super().__init__()
        
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.a = nn.Linear(2 * out_features, 1, bias=False)
        self.leaky_relu = nn.LeakyReLU(0.2)
        
    def forward(self, x, adj, return_attention=False):
        N = x.size(0)
        h = self.W(x)  # [N, out_features]
        
        # 計算注意力分數
        h_repeat_i = h.unsqueeze(1).expand(-1, N, -1)  # [N, N, F]
        h_repeat_j = h.unsqueeze(0).expand(N, -1, -1)  # [N, N, F]
        concat = torch.cat([h_repeat_i, h_repeat_j], dim=-1)  # [N, N, 2F]
        
        e = self.leaky_relu(self.a(concat).squeeze(-1))  # [N, N]
        
        # 遮罩
        adj_with_self = adj + torch.eye(N).to(adj.device)
        e = e.masked_fill(adj_with_self == 0, float('-inf'))
        
        attention = F.softmax(e, dim=1)  # [N, N]
        
        h_prime = torch.mm(attention, h)  # [N, out_features]
        
        if return_attention:
            return h_prime, attention
        return h_prime

def visualize_attention(model, x, adj, node_labels=None):
    """
    可視化 GAT 的注意力權重
    """
    model.eval()
    
    with torch.no_grad():
        _, attention = model(x, adj, return_attention=True)
    
    attention = attention.cpu().numpy()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 注意力矩陣
    im = axes[0].imshow(attention, cmap='hot')
    axes[0].set_title('注意力權重矩陣')
    axes[0].set_xlabel('目標節點 j')
    axes[0].set_ylabel('源節點 i')
    plt.colorbar(im, ax=axes[0])
    
    # 圖可視化（邊粗細表示注意力）
    G = nx.from_numpy_array(adj.cpu().numpy())
    pos = nx.spring_layout(G, seed=42)
    
    # 繪製節點
    if node_labels is not None:
        colors = ['#FF6B6B' if l == 0 else '#4ECDC4' for l in node_labels]
    else:
        colors = '#4ECDC4'
    
    nx.draw_networkx_nodes(G, pos, ax=axes[1], node_color=colors, node_size=300)
    nx.draw_networkx_labels(G, pos, ax=axes[1], font_size=8)
    
    # 繪製邊（粗細與注意力相關）
    for i, j in G.edges():
        weight = (attention[i, j] + attention[j, i]) / 2  # 對稱化
        nx.draw_networkx_edges(G, pos, ax=axes[1], edgelist=[(i, j)],
                              width=weight * 5, alpha=max(0.2, weight))
    
    axes[1].set_title('圖可視化（邊粗細 = 注意力權重）')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

# 測試
gat_vis = GATWithAttention(8, 16)
visualize_attention(gat_vis, node_features, adj_matrix)

---
## Part 6: 進階 GNN 技巧與實務

### 6.1 過度平滑問題 (Over-smoothing)

當 GNN 層數增加時，所有節點的表示會越來越相似，最終趨於同一個值。這是 GNN 的一個重要限制。

In [ ]:
# ========== 過度平滑問題 ==========

def measure_smoothness(x):
    """
    測量節點表示的平滑度（越小越相似）
    使用節點表示間的平均距離
    """
    n = x.size(0)
    x_norm = F.normalize(x, p=2, dim=1)
    
    # 計算所有節點對的相似度
    similarity = torch.mm(x_norm, x_norm.t())
    
    # 排除自己
    mask = ~torch.eye(n, dtype=torch.bool).to(x.device)
    avg_similarity = similarity[mask].mean().item()
    
    return avg_similarity

def demonstrate_oversmoothing(x, adj, max_layers=10):
    """
    展示 GNN 層數增加時的過度平滑問題
    """
    results = {'layers': [], 'smoothness': []}
    
    # 初始平滑度
    h = x.clone()
    results['layers'].append(0)
    results['smoothness'].append(measure_smoothness(h))
    
    # 簡單的 GCN propagation
    adj_hat = adj + torch.eye(adj.size(0)).to(adj.device)
    degree = adj_hat.sum(dim=1)
    d_inv_sqrt = torch.diag(degree.pow(-0.5))
    adj_norm = torch.mm(torch.mm(d_inv_sqrt, adj_hat), d_inv_sqrt)
    
    for layer in range(1, max_layers + 1):
        h = torch.mm(adj_norm, h)
        h = F.relu(h)
        
        results['layers'].append(layer)
        results['smoothness'].append(measure_smoothness(h))
    
    # 可視化
    plt.figure(figsize=(10, 4))
    plt.plot(results['layers'], results['smoothness'], 'b-o', linewidth=2)
    plt.xlabel('GNN 層數')
    plt.ylabel('節點表示相似度 (越高越平滑)')
    plt.title('Over-smoothing: 層數增加導致節點表示趨同')
    plt.grid(True, alpha=0.3)
    plt.axhline(y=1.0, color='r', linestyle='--', label='完全相同')
    plt.legend()
    plt.show()
    
    print("💡 觀察：隨著層數增加，節點表示越來越相似")
    print("   這就是為什麼大多數 GNN 只用 2-3 層")

# 展示過度平滑
demonstrate_oversmoothing(node_features, adj_matrix, max_layers=15)

In [ ]:
# ========== 緩解 Over-smoothing 的方法 ==========

class ResidualGCN(nn.Module):
    """
    方法 1: 殘差連接 (Residual Connection)
    
    h' = h + GCN(h)
    保留原始資訊，避免完全被鄰居同化
    """
    
    def __init__(self, in_features, hidden_features, out_features, n_layers=5):
        super().__init__()
        
        self.input_proj = nn.Linear(in_features, hidden_features)
        
        self.layers = nn.ModuleList([
            GCNLayer(hidden_features, hidden_features) for _ in range(n_layers)
        ])
        
        self.output_proj = nn.Linear(hidden_features, out_features)
        
    def forward(self, x, adj):
        x = self.input_proj(x)
        
        for layer in self.layers:
            x_new = layer(x, adj)
            x = x + F.relu(x_new)  # 殘差連接
        
        return self.output_proj(x)


class JumpingKnowledgeGCN(nn.Module):
    """
    方法 2: Jumping Knowledge (JK)
    
    聚合所有層的輸出，而不只用最後一層
    可以使用 concat、max 或 LSTM 聚合
    """
    
    def __init__(self, in_features, hidden_features, out_features, n_layers=5, mode='concat'):
        super().__init__()
        
        self.mode = mode
        self.n_layers = n_layers
        
        self.layers = nn.ModuleList()
        self.layers.append(GCNLayer(in_features, hidden_features))
        
        for _ in range(n_layers - 1):
            self.layers.append(GCNLayer(hidden_features, hidden_features))
        
        if mode == 'concat':
            self.output_proj = nn.Linear(hidden_features * n_layers, out_features)
        else:  # max
            self.output_proj = nn.Linear(hidden_features, out_features)
    
    def forward(self, x, adj):
        layer_outputs = []
        
        for layer in self.layers:
            x = F.relu(layer(x, adj))
            layer_outputs.append(x)
        
        # 聚合所有層
        if self.mode == 'concat':
            x = torch.cat(layer_outputs, dim=-1)
        else:  # max
            x = torch.stack(layer_outputs, dim=0).max(dim=0)[0]
        
        return self.output_proj(x)


class DropEdgeGCN(nn.Module):
    """
    方法 3: DropEdge
    
    訓練時隨機丟棄邊，類似 Dropout
    減緩訊息傳播，避免過度平滑
    """
    
    def __init__(self, in_features, hidden_features, out_features, n_layers=5, drop_rate=0.3):
        super().__init__()
        
        self.drop_rate = drop_rate
        
        self.layers = nn.ModuleList()
        self.layers.append(GCNLayer(in_features, hidden_features))
        
        for _ in range(n_layers - 1):
            self.layers.append(GCNLayer(hidden_features, hidden_features))
        
        self.output_proj = nn.Linear(hidden_features, out_features)
    
    def drop_edge(self, adj):
        """隨機丟棄邊"""
        if not self.training:
            return adj
        
        mask = torch.rand_like(adj) > self.drop_rate
        return adj * mask.float()
    
    def forward(self, x, adj):
        for layer in self.layers:
            adj_dropped = self.drop_edge(adj)
            x = F.relu(layer(x, adj_dropped))
        
        return self.output_proj(x)


# 測試並比較
print("緩解 Over-smoothing 的方法：")
print("\n1. 殘差連接 (ResGCN):")
res_gcn = ResidualGCN(8, 16, 2, n_layers=5)
out = res_gcn(node_features, adj_matrix)
print(f"   5 層 ResGCN 輸出形狀: {out.shape}")

print("\n2. Jumping Knowledge:")
jk_gcn = JumpingKnowledgeGCN(8, 16, 2, n_layers=5, mode='concat')
out = jk_gcn(node_features, adj_matrix)
print(f"   5 層 JK-GCN 輸出形狀: {out.shape}")

print("\n3. DropEdge:")
drop_gcn = DropEdgeGCN(8, 16, 2, n_layers=5, drop_rate=0.3)
out = drop_gcn(node_features, adj_matrix)
print(f"   5 層 DropEdge-GCN 輸出形狀: {out.shape}")

### 6.2 連結預測 (Link Prediction)

連結預測是 GNN 的重要應用：預測兩個節點之間是否應該存在邊。

應用場景：
- 社交網路：好友推薦
- 知識圖譜：關係補全
- 生物網路：蛋白質交互作用預測

---
## 🎯 總結：GNN 完整知識框架

### 📊 本模組重點回顧

| 主題 | 核心概念 | 關鍵公式/技巧 |
|------|----------|---------------|
| **圖表示** | 鄰接矩陣、邊列表 | $A_{ij} = 1$ 表示連接 |
| **訊息傳遞** | MSG → AGG → UPDATE | 每層聚合 1 跳鄰居 |
| **GCN** | 對稱歸一化 | $H' = \sigma(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}HW)$ |
| **GAT** | 注意力加權 | $\alpha_{ij} = \text{softmax}(e_{ij})$ |
| **GraphSAGE** | 可學習聚合 | $h' = W_1h + W_2 \cdot \text{AGG}(N)$ |
| **Over-smoothing** | 深層 GNN 問題 | 殘差、JK、DropEdge |
| **連結預測** | 節點對評分 | $\sigma(z_i^T z_j)$ 或 MLP |

### 🔄 GNN 方法深度比較

```
┌────────────┬─────────────────┬─────────────────┬─────────────────┐
│ 方法       │ 聚合方式        │ 優點            │ 缺點            │
├────────────┼─────────────────┼─────────────────┼─────────────────┤
│ GCN        │ 度歸一化平均    │ 簡單高效        │ 固定權重        │
│ GAT        │ 注意力加權      │ 可學習重要性    │ 計算量大        │
│ GraphSAGE  │ 分離自己/鄰居   │ 歸納學習        │ 需採樣          │
│ GIN        │ Sum + MLP       │ 理論最強表達    │ 可能過擬合      │
│ MPNN       │ 通用框架        │ 靈活性高        │ 設計空間大      │
└────────────┴─────────────────┴─────────────────┴─────────────────┘
```

### 🛠️ GNN 實戰問題診斷

```
問題 1: 過度平滑 (Over-smoothing)
  症狀：深層 GNN 性能下降，節點表示趨同
  解決：
    - 減少層數（通常 2-3 層最佳）
    - 殘差連接（ResGCN）
    - Jumping Knowledge
    - DropEdge / DropNode

問題 2: 訓練數據不平衡
  症狀：少數類別性能差
  解決：
    - 重新採樣
    - 類別權重
    - Focal Loss

問題 3: 大規模圖處理困難
  症狀：內存溢出，速度慢
  解決：
    - Mini-batch 訓練（鄰居採樣）
    - GraphSAGE 風格採樣
    - Cluster-GCN
    - 分佈式訓練
```

### 📝 GNN 訓練 Checklist

**模型設計**：
- [ ] 確定任務類型（節點/邊/圖級別）
- [ ] 選擇合適的 GNN 變體
- [ ] 設定層數（通常 2-3 層）
- [ ] 選擇聚合函數（mean, sum, max）

**數據處理**：
- [ ] 構建圖結構（鄰接矩陣/邊列表）
- [ ] 設計節點特徵
- [ ] 處理自環（是否添加）
- [ ] 訓練/驗證/測試分割

**訓練技巧**：
- [ ] 使用適當的正則化（Dropout, DropEdge）
- [ ] 監控過度平滑指標
- [ ] 對於大圖使用採樣
- [ ] 早停 (Early Stopping)

### 🔗 常見 GNN 應用場景

| 任務類型 | 應用場景 | 推薦方法 |
|----------|----------|----------|
| **節點分類** | 社交網路用戶分類 | GCN, GAT |
| **連結預測** | 好友推薦、藥物交互 | GraphSAGE + Decoder |
| **圖分類** | 分子性質預測 | GIN + Pooling |
| **節點聚類** | 社區發現 | DGI, GRACE |
| **圖生成** | 藥物發現 | GraphVAE, GraphRNN |

### 🔗 延伸學習資源

| 資源 | 說明 |
|------|------|
| [GCN 原論文](https://arxiv.org/abs/1609.02907) | Kipf & Welling, 2016 |
| [GAT 原論文](https://arxiv.org/abs/1710.10903) | Veličković et al., 2017 |
| [GraphSAGE](https://arxiv.org/abs/1706.02216) | Hamilton et al., 2017 |
| [GIN 論文](https://arxiv.org/abs/1810.00826) | Xu et al., 2018 |
| [PyG 文檔](https://pytorch-geometric.readthedocs.io/) | PyTorch Geometric |
| [Stanford CS224W](http://web.stanford.edu/class/cs224w/) | 圖機器學習課程 |

### ⏭️ 下一步

在下一個模組中，我們將學習 **性能優化與部署**：
- DataLoader 優化（num_workers, pin_memory）
- 混合精度訓練（AMP）
- 模型量化與剪枝
- 模型導出與部署（TorchScript, ONNX）

---
## 總結

### 本模組重點

1. **圖的基本概念**
   - 節點、邊、鄰接矩陣
   - 節點特徵表示

2. **訊息傳遞機制**
   - 訊息 → 聚合 → 更新
   - 每一層聚合一跳鄰居

3. **GCN**
   - 對稱歸一化的鄰接矩陣
   - $H' = \sigma(\hat{A} H W)$

4. **GAT**
   - 注意力機制分配鄰居權重
   - 多頭注意力增強表達能力

### GNN 方法比較

```
┌──────────┬────────────────┬────────────────┐
│ 方法     │ 聚合方式       │ 特點           │
├──────────┼────────────────┼────────────────┤
│ GCN      │ 平均 + 歸一化  │ 簡單高效       │
│ GAT      │ 注意力加權     │ 可學習權重     │
│ GraphSAGE│ 可選聚合函數   │ 歸納學習       │
│ GIN      │ sum + MLP      │ 理論保證       │
└──────────┴────────────────┴────────────────┘
```

### 常見應用

- **節點分類**：社交網路用戶分類
- **連結預測**：推薦系統
- **圖分類**：分子性質預測
- **圖生成**：藥物發現

### 下一步

在下一個模組中，我們將學習**性能優化與部署**，包括 DataLoader 優化、混合精度訓練、模型保存與載入。